Read training data list

In [0]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
drive.mount('/content/drive')
default_path = '/content/drive/My Drive/INF ML 552/HW5 - Neural Networks/'
# training_list = pd.read_csv(default_path + 'downgesture_train.list.txt', header=None, sep=',')
# training_list

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read training data

In [0]:
# X = input data (pixel_array)
# y = output (labels)
def input_output(file):
    input_X = []
    output_y = []
    with open(default_path + file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if "down" in line:
                output_y.append(1)
            else:
                output_y.append(0)
                
            img_array = mpimg.imread(default_path + line)
            # normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
            img_array = img_array.astype('float32')
            img_array = img_array / 255.0
            input_X.append(img_array)
    return input_X, output_y

In [0]:
train_X, train_y = input_output('downgesture_train.list.txt')
test_X, test_y = input_output('downgesture_test.list.txt')

In [0]:
for i in range(len(train_X)):
  train_X[i] = train_X[i].flatten()
train_X = pd.DataFrame(train_X)


In [0]:
train_X

In [0]:
train_y = pd.DataFrame(train_y)

In [0]:
train_y

0    0
Name: 179, dtype: int64

Main neural network function

In [0]:
# build the network
#         w_ij(l)/b1    w_ij(l+1)/b2   
#184(inputs) -----> 100 -------> 1(output)
#   x_i(l-1)  s_j(l)  x_j(l) s_j(l+1)  x_j(l+1)=y
learning_rate = 0.1
epochs = 1000
weights =[[],[]]
weights[0] = [get_weights(100) for i in range(961)]
weights[1] = [get_weights(1) for i in range(101)]
N = train_X.shape[0]
input_size = train_X.shape[1]
hidden_layer_size = 100
output_size = 1
level = [input_size,hidden_layer_size,output_size]

Get n random weights between -0.01 to 0.01

In [0]:
def get_weights(n):
  w_ij = []
  for i in range(n):
    i = np.random.uniform(-0.01,0.01)
    w_ij.append(i)
  return w_ij

Feed-forward

In [0]:
def sigmoid(s):
  # print(s)
  return 1/(1 + np.exp(-s))

def deriv_sigmoid(s):
  return sigmoid(s) * (1 - sigmoid(s))

def feed_forward(w,input_x):

  x= []
  # print(input_x)
  x.append(list(input_x))
  x[0].insert(0,1)
  x.append([0] * hidden_layer_size)
  # x[1].insert(0,1)
  x.append([0] * output_size)

  for l in range(1,3):
    for j in range(level[l]):
      for i in range(len(x[l-1])):
        x[l][j] += w[l-1][i][j] * x[l-1][i]
      x[l][j] = sigmoid(x[l][j])
    x[l].insert(0,1)
  
  x[2] = x[2][1:]
  return x



In [0]:
def get_delta(x,w,y):

  delta = [[0]]
  delta[0][0] = 2 * (x[2][0] - y) * (1 - math.pow(x[2][0],2))
  delta.insert(0,[0] * 100)
  for i in range(1,100):
    delta[0][i] = 1 - math.pow(x[1][i+1],2) * w[1][i+1][0] * delta[1][0]
  return delta


In [0]:
len(delta[0])

In [0]:
def step3_weights(x,w,delta,lr):
  for l in range(0,2):
    for j in range(len(delta[l])):
      for i in range(len(x[l])):
        w[l][i][j] -= lr *  x[l][i] * delta[l][j]
  return w

weights2 = step3_weights(lala,weights,delta, learning_rate)

In [0]:
def ffbpnn(w,lr):
  ep = 0
  for a in range(5):
    for i in range(N):
      print(ep)
      x = feed_forward(w, train_X.iloc[i])
      delta = get_delta(x,w,train_y.iloc[i][0])
      w = step3_weights(x,w,delta,lr)
      ep += 1
  
  for i in range(80):
      print(ep)
      x = feed_forward(w, train_X.iloc[i])
      delta = get_delta(x,w,train_y.iloc[i][0])
      w = step3_weights(x,w,delta,lr)
      ep += 1
  
  return w

w = ffbpnn(weights, learning_rate)

In [0]:

pd.DataFrame(w).to_csv(default_path + 'weights.csv', index=False)

In [0]:
for i in range(len(test_X)):
  test_X[i] = test_X[i].flatten()
test_X = pd.DataFrame(test_X)

In [0]:
def testing(test_X,weights):
  y_pred = test_X.apply(lambda z : feed_forward(weights,z)[2][0], axis=1)
  return y_pred



In [0]:
test_y = pd.DataFrame(test_y)

In [0]:
y_pred = testing(test_X, w)
y_pred= pd.DataFrame(y_pred)
y_pred = y_pred.rename(columns={0: 'prediction'})
y_pred['label'] = test_y[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
y_pred['final'] = y_pred['prediction'].apply(lambda z: (0 if z < 0.5 else 1))

In [0]:
y_pred['compare'] = y_pred['label'] == y_pred['final']

In [0]:
sum(y_pred['compare'])/y_pred.shape[0]

0.7710843373493976

In [0]:
y_pred.to_csv(default_path + 'prediction.csv', index=False)

Part 2

In [0]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100), activation='logistic',solver='sgd', 
                    learning_rate='constant',learning_rate_init=0.1,max_iter=1000)
mlp.fit(train_X, train_y.values.ravel())

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=100, learning_rate='constant',
              learning_rate_init=0.1, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
for i in range(len(test_X)):
    test_X[i] = test_X[i].flatten()
test_X = pd.DataFrame(test_X)

test_y = pd.DataFrame(test_y)

In [0]:
predictions = mlp.predict(test_X)
predictions

array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(test_y,predictions))
print(classification_report(test_y,predictions))

[[60  4]
 [ 5 14]]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93        64
           1       0.78      0.74      0.76        19

    accuracy                           0.89        83
   macro avg       0.85      0.84      0.84        83
weighted avg       0.89      0.89      0.89        83

